In [82]:
from keras import layers
import tensorflow as tf
import pandas as pd
import numpy as np

# Make numpy values easier to read.
np.set_printoptions(precision=3, suppress=True)


df = pd.read_csv('../csv_dateien/starcraftFinalcsvs/allRunsButOnlyTerran.csv')

conditions = [
    (df['fraction'] == "Protoss"), #1
    (df['fraction'] == "Terraner"), #2
    (df['fraction'] == "Zerg"), #3
    (df['fraction'] == "T"), #2
]

values = [1, 2, 3, 2]

df["fraction"] = np.select(conditions, values)
df=df[df["total_army"]>0]
df=df[df["fraction"]==2]
df=df[df["supply"]<=200]
df["ratio"] = df["total_workers"]/df["total_army"]

dfPrepSample = df.drop(columns=["player","winner","replay_filename", "Unnamed: 0", "total_workers", "total_army_value","fraction"])
len(dfPrepSample)


3009

In [83]:
df_sample = dfPrepSample.sample(frac=1, random_state=12)
dfTrainSample = df_sample.sample(frac=0.8)

dfTrainSample.head()

,resource_mining,total_army,supply,time,ratio
5181,8838,21.0,54.0,441,1.380952
3754,55199,103.0,197.0,1058,0.514563
3850,7698,14.0,70.0,411,2.714286
4259,36813,91.0,165.0,982,0.813187
44,6382,10.0,46.0,337,3.100000


In [84]:
dfTestSample = df_sample.drop(dfTrainSample.index)

dfTestSample.head()

,resource_mining,total_army,supply,time,ratio
4458,18014,44.0,110.0,617,1.113636
3646,20336,67.0,157.0,759,1.000000
1117,9124,17.0,70.0,472,2.294118
3241,14496,85.0,150.0,724,0.517647
5721,20919,40.0,109.0,652,1.450000


In [85]:
dfTrainFeatures = dfTrainSample.copy()
dfTrainFeatures

,resource_mining,total_army,supply,time,ratio
5181,8838,21.0,54.0,441,1.380952
3754,55199,103.0,197.0,1058,0.514563
3850,7698,14.0,70.0,411,2.714286
4259,36813,91.0,165.0,982,0.813187
44,6382,10.0,46.0,337,3.100000
...,...,...,...,...,...
3980,34776,73.0,165.0,773,0.904110
5477,22982,62.0,157.0,766,0.935484
3615,24385,18.0,77.0,571,2.944444
4698,14997,55.0,102.0,608,0.818182


In [86]:

dfTrainLabels = dfTrainFeatures.pop("total_army")
dfTrainFeatures = np.array(dfTrainFeatures)
dfTrainFeatures

array([[ 8838.   ,    54.   ,   441.   ,     1.381],
       [55199.   ,   197.   ,  1058.   ,     0.515],
       [ 7698.   ,    70.   ,   411.   ,     2.714],
       ...,
       [24385.   ,    77.   ,   571.   ,     2.944],
       [14997.   ,   102.   ,   608.   ,     0.818],
       [42764.   ,   148.   ,   775.   ,     1.404]])

In [87]:
# normalize = layers.Normalization()
# normalize.adapt(dfTrainFeatures)

In [88]:
sc_model = tf.keras.Sequential([layers.Dense(3, activation="relu"),layers.Dense(3, activation="relu"),layers.Dense(3, activation="relu"), layers.Dense(1)]) # letzte lösung 30/5

sc_model.compile(loss = tf.keras.losses.MeanSquaredError(),
                      optimizer = tf.keras.optimizers.Adam())

In [89]:
sc_model.fit(dfTrainFeatures, dfTrainLabels, epochs=1000)  #350

Epoch 1/1000
76/76 [==============================] - 0s 624us/step - loss: 1079370.1250
Epoch 2/1000
76/76 [==============================] - 0s 597us/step - loss: 305053.5312
Epoch 3/1000
76/76 [==============================] - 0s 602us/step - loss: 128147.9453
Epoch 4/1000
76/76 [==============================] - 0s 590us/step - loss: 66262.3359
Epoch 5/1000
76/76 [==============================] - 0s 594us/step - loss: 37686.6016
Epoch 6/1000
76/76 [==============================] - 0s 611us/step - loss: 22760.8809
Epoch 7/1000
76/76 [==============================] - 0s 608us/step - loss: 14228.5996
Epoch 8/1000
76/76 [==============================] - 0s 642us/step - loss: 9075.5068
Epoch 9/1000
76/76 [==============================] - 0s 638us/step - loss: 5872.5020
Epoch 10/1000
76/76 [==============================] - 0s 600us/step - loss: 3840.5718
Epoch 11/1000
76/76 [==============================] - 0s 593us/step - loss: 2540.0889
Epoch 12/1000
76/76 [====================

In [90]:
sc_model.summary()


Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_28 (Dense)            (None, 3)                 15        
                                                                 
 dense_29 (Dense)            (None, 3)                 12        
                                                                 
 dense_30 (Dense)            (None, 3)                 12        
                                                                 
 dense_31 (Dense)            (None, 1)                 4         
                                                                 
Total params: 43
Trainable params: 43
Non-trainable params: 0
_________________________________________________________________


In [91]:
dfTestFeatures = dfTestSample.copy()

dfTestLabels = dfTestFeatures.pop("total_army")
dfTestFeatures = np.array(dfTestFeatures)
dfTestLabels.head()

4458    44.0
3646    67.0
1117    17.0
3241    85.0
5721    40.0
Name: total_army, dtype: float64

In [92]:
from sklearn.metrics import r2_score


predictedValues = sc_model.predict(dfTestFeatures)
r2_score_sc = (r2_score(dfTestLabels, predictedValues))
stri = ""
if(r2_score_sc<0):
    stri = "negative" + str(r2_score_sc)
else:
    stri = str(r2_score_sc)

print(f"The R^2 value in the test set is {stri}")

19/19 [==============================] - 0s 579us/step
The R^2 value in the test set is 0.9234082563107449


In [37]:
sc_model.save('sc_ts_model.h5')
